# State Machine and Object Graph Programming

* (c) Dmitry Ponyatov <<dponyatov@gmail.com>>
* (c) Valeri Rakitine https://www.linkedin.com/in/valrakitine/

### Install (Linux)

```
~$ git clone -o gh https://github.com/ponyatov/OGP.git
~$ cd OGP
~/OGP$ make install
```
### Run
```
~/OGP$ make jupyter
```